In [13]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import time


%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [ ]:
%%cython
import numpy as np
import igraph as ig
import time


def update_node_state(g, int[:] state, int[:] old_state, int vertex_id, double mu, double beta):
    cdef int infected_vertices
    cdef int v
    if old_state[vertex_id]:
        state[vertex_id] = np.random.binomial(1, mu)

    else:
        infected_vertices = sum([old_state[v] for v in g.neighbors(vertex_id)])
        state[vertex_id] = np.random.binomial(1, 1 - (1 - beta)**infected_vertices)

    return state[vertex_id]



def simulation(g, int len_g, int[:] state, int[:] old_state, double mu, double beta):
    
    # Constants
    cdef double P0 = 0.2
    cdef int TMAX = 1000
    cdef int TSKIP = 100
    cdef double[:] infected_ratio_list = np.zeros(TMAX - TSKIP)
    cdef long start_time
    cdef int i, j
    
    start_time = time.time()
    #simulation loop
    for i in range(TMAX):
        start_time = time.time()
        
        # update old state
        for j in range(len_g):
            old_state[j] = state[j]
        
        # To skip the saving of transionary states
        if i >= TSKIP:
            for j in range(len_g):
                infected_ratio_list[i - TSKIP] += update_node_state(g, state, old_state, j, mu, beta)
            infected_ratio_list[i - TSKIP] /= len_g
        else:
            for j in range(len_g):
                update_node_state(g, state, old_state, j, mu, beta)
        
    return np.array(infected_ratio_list)

In [7]:
MU = 0.5
BETA = 0.5
P0 = 0.2

In [8]:
len_g = 1000
g = ig.Graph.Barabasi(len_g, 5)

In [17]:
state = np.array([np.random.binomial(1, P0) for _ in range(len_g)])
old_state = state[:]

for i in range(100):
    st = time.time()
    a = simulation(g, len_g, state, old_state, MU, BETA)
    print("Iteration number {} requires {} seconds".format(i, time.time()-st))
    state = np.array([np.random.binomial(1, P0) for _ in range(len_g)])
    old_state = state[:]  

Iteration number 0 requires 5.2865118980407715 seconds
Iteration number 1 requires 4.855099439620972 seconds
Iteration number 2 requires 4.968764543533325 seconds
Iteration number 3 requires 4.889921426773071 seconds
Iteration number 4 requires 4.756247282028198 seconds
Iteration number 5 requires 4.950589418411255 seconds
Iteration number 6 requires 4.957923173904419 seconds
Iteration number 7 requires 5.101411581039429 seconds
Iteration number 8 requires 4.770010471343994 seconds
Iteration number 9 requires 5.181143760681152 seconds
Iteration number 10 requires 4.889872074127197 seconds
Iteration number 11 requires 4.96875 seconds
Iteration number 12 requires 5.082493782043457 seconds
Iteration number 13 requires 4.960771799087524 seconds
Iteration number 14 requires 4.778923034667969 seconds
Iteration number 15 requires 4.920834302902222 seconds
Iteration number 16 requires 4.862030982971191 seconds
Iteration number 17 requires 4.863071918487549 seconds
Iteration number 18 requires 

KeyboardInterrupt: 